# E-SC4R: Explaining Software Clustering for Remodularisation

- Tan, Alvin Jian Jia, Chun Yong Chong, and Aldeida Aleti. "E-SC4R: Explaining Software Clustering for Remodularisation." Journal of Systems and Software 186 (2022): 111162.

# REARRANGE: An Effort Estimation Approach for Software Clustering-based Remodularisation

## Abstract

Software clustering is often used as a remodularisation technique to suggest ways to improve the internal quality of the software through some suggested refactoring operations. This project aims to provide an end-to-end pipeline to help developers in carrying out refactoring activities through the following steps:

1. Determine the most suitable clustering algorithm for a given project. (**E-SC4R**)
2. Estimate the effort needed to convert the current project structure to the suggested clustering result. (**REARRANGE**)

## Introduction

This repo contains scripts and notebooks that are used in both E-SC4R and REARRANGE. The following snippets show the general flow of the end-to-end pipeline. For more details and the full implementation, kindly refer back to the specific notebooks.

# E-SC4R
<img src="README images/E-SC4R framework.PNG" width="800" height="400">
<img src="README images/SoftwareClusteringProcess.PNG" width="800" height="400">


## E-SC4R Experiment Design
1. Idenfitication of clustering entities
2. Idenfitication of clustering features
3. Calculation of similarity measure
4. Application of clustering algorithm
5. Evaluation of clustering Results

In [7]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
import requests
from github import Github
from git import Repo
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp
import configparser
import h2o
import scipy as sp

### Identification of clustering entitites

This step explores the file directory structure and extracts the clustering entities, child and parent classes.

In [11]:
project_name = 'Redisson'
version_name = 'redisson-3.16.0'


n_cluster = 300
affinity = 'cosine'
linkage = 'single'
rootdir = f'C:/Users/tanji/Desktop/SoftwareRemodularization/raw_sourcecode/{project_name}/{project_name}_{version_name}'
full_dir_arr = []
for root, dirs, files in os.walk(rootdir):
    #print(root)
    #print(dirs)
    for element in files:
        if '.java' in element:
            dir_string = root + '\\' + element
            full_dir_arr.append(dir_string)

cluster_dict = {}
cluster_tree = {}


for element in full_dir_arr:
    element = element.split('\\')
    child = element[-1]
    parent = element[-2]
    cluster_tree[child] = parent
    
cluster_tree

{'ElementsSubscribeService.java': 'redisson',
 'JndiRedissonFactory.java': 'redisson',
 'MapWriteBehindTask.java': 'redisson',
 'MapWriterTask.java': 'redisson',
 'PubSubEntry.java': 'handler',
 'PubSubMessageListener.java': 'redisson',
 'PubSubPatternMessageListener.java': 'redisson',
 'PubSubPatternStatusListener.java': 'redisson',
 'PubSubStatusListener.java': 'redisson',
 'QueueTransferService.java': 'redisson',
 'QueueTransferTask.java': 'redisson',
 'RedisClusterNodes.java': 'redisson',
 'RedisNodes.java': 'redisnode',
 'Redisson.java': 'redisson',
 'RedissonAtomicDouble.java': 'redisson',
 'RedissonAtomicLong.java': 'redisson',
 'RedissonBaseAdder.java': 'redisson',
 'RedissonBaseLock.java': 'redisson',
 'RedissonBatch.java': 'redisson',
 'RedissonBinaryStream.java': 'redisson',
 'RedissonBitSet.java': 'redisson',
 'RedissonBlockingDeque.java': 'redisson',
 'RedissonBlockingQueue.java': 'redisson',
 'RedissonBloomFilter.java': 'redisson',
 'RedissonBoundedBlockingQueue.java': 'r

In [ ]:
cluster_tree